In [1]:
import torch
import torchvision
from torchvision import transforms, datasets 
from torch import nn, optim
import torch.nn.functional as F

from models.model import Net

### データセット

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# training dataset
trainset = datasets.CIFAR10(root='./data', train=True,
                                        download=False, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=False, num_workers=2)

# test dataset
testset = datasets.CIFAR10(root='./data', train=False,
                                       download=False, transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

### 学習してパラメータの保存

In [3]:
# ネットワーク, 損失関数, 最適化
model_train = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_train.parameters(), lr=0.001, momentum=0.9)

# ========== 学習 ==========
for epoch in range(2):
    
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        
        # 学習
        inputs, labels = data  # データの取得
        optimizer.zero_grad()  # 勾配の初期化
        outputs = model_train(inputs)  # 順伝搬
        loss = criterion(outputs, labels)  # 誤差の計算
        loss.backward()   # 逆伝搬
        optimizer.step()  # 重み更新
        
        # 状態の表示
        running_loss += loss.item()
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
            
print('Finished Training')


# ========== 保存 ==========
torch.save(model_train.state_dict(), "./params/model.pth")

[1,  2000] loss: 2.216
[1,  4000] loss: 1.864
[1,  6000] loss: 1.677
[1,  8000] loss: 1.570
[1, 10000] loss: 1.545
[1, 12000] loss: 1.483
[2,  2000] loss: 1.420
[2,  4000] loss: 1.398
[2,  6000] loss: 1.343
[2,  8000] loss: 1.306
[2, 10000] loss: 1.308
[2, 12000] loss: 1.268
Finished Training


### 学習したパラメータの読み込み

In [5]:
# ネットワーク再定義（検証のため）
model_test = Net()

# loading前のネットワークでテストしてみる
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model_test(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
print('===== before loading =====')
print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))


===== before loading =====
Accuracy of the network on the 10000 test images: 11 %


In [7]:
# === ロード ===
model_test.load_state_dict(torch.load("./params/model.pth"))


# loading後のネットワークでテストしてみる
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model_test(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
print('===== after loading =====')
print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))


===== after loading =====
Accuracy of the network on the 10000 test images: 56 %
